# Lecture 4 : Backpropagation and Neural Networks
Lecture 4 slides : http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture4.pdf<br>
Lecture 4 vidoe : https://www.youtube.com/watch?v=d14TUNcbn1k&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=4

<font size=4>$s=f(s;W) = Wx$</font> &emsp; <font color='blue'>scores function</font> <br>
<font size=4>$L_i = \sum_{j\neq y_i} max(0,s_j- s_{y_i}+1)$ </font>&emsp; <font color='blue'>SVM loss</font><br>
<font size=4>$L = \frac{1}{N} \sum_{i=1}^N L_i + \sum_k W_k^2$ </font> &emsp; <font color='blue'>Data Loss + Regularization</font><br>
<font size=4>want $\color{orange} {\nabla_W L} $ </font><br>

In [1]:
%%HTML
<style> code {background-color : #C9EBF9 !important;} </style>

```python
# Vanilla Gradient Descent
while True:
    weights_grad = evaluate_gradient(loss_func, data, weights)
    weight += -step_size*weight_Grad #perform parameter update
```

<font size=6>Gradient descent</font><br><br>
<font size=4>$\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h) - f(x)}{h}$</font><br>
<b>Numerical gradient</b>:slow, approximate, easy to write<br>
<b>Analytic gradient</b>: fast, exact, error-prone<br>
In practice : Derive analytic gradient, check your implementation with numerical gradient

<img src='./Lesson pic/4-8.png'>
<img src='./Lesson pic/4-23.png'>
<img src='./Lesson pic/4-29.png'>

leverage 이점
ex) leverage of chain rule

<img src='./Lesson pic/4-45.png'>
<img src='./Lesson pic/4-50.png'>

add gate : 같은 값을 분산하는 gradient distributor<br>
max gate : z에는 1, w에는 0인 gradient router <br>
mul gate : gradient로 다른 변수의 값을 가진다.gradient switcher<br>
ex) f=xy일 때 y로의 backpropagatioin = [local gradient]x[upstream gradient] <br>
= $\frac {df}{dy} * [upstream gradient] = x * (-2) = 3 * (-2) = -6$<br>



<font size=5>$\frac{\partial f}{\partial x} = \sum_i \frac{\partial f}{\partial q_i} \frac{\partial q_i}{\partial x}$</font>

<img src='./Lesson pic/4-52.png'>
<img src='./Lesson pic/4-56.png'>
<img src='./Lesson pic/4-57.png'>

## Jacobian Matrix is diagonal Matrix

<font size=4>$f(x,W) => \{x \Subset R^n | W \Subset \mathbb{R}^{n \times n}\}$</font>

<img src='./Lesson pic/4-65.png'>
<img src='./Lesson pic/4-70.png'>
<img src='./Lesson pic/4-74.png'>

# Modularized implementation: forward / backward API

Graph (or Net) object (rough psuedo code)
```python
class ComputationalGraph(object):
    def forward(inputs):
        # 1. [pass inputs to input gate]
        # 2. forward the computational graph:
        for gate in self.graph.nodes_topologically_sorted():
            gate.forward()
        return loss
    
    def backward():
        for gate in self.graph.nodes_topologically_sorted():
            gate.backward() #littel piece of backprop(chain rule applied)
        return inputs_gradients
```

## ex) z = x * y
```python
class MultiplyGate(object):
    def forward(x,y):
        z=x*y
        self.x=x
        self.y=y
        return z
    def backward(dz):
        dx = self.y * dz #[dz/dx * dL/dz]
        dy = self.x * dz #[dz/dy * dL/dz]
        return [dx,dy]
```

# Summary so far - Slides 4 - 81
 - <font size=4>neural nets will be very large : impractical to write down gradient formula by hand for all parameters
 - <b>backpropagation</b> = recursive application of the chain rule along a computational graph to compute the gradients of all inputs/ parameters/intermediates
 - implementations maintain a graph structure, where the nodes implement the <b>forward()/backward()</b> API
 - <b>Forward</b>: compute result of an operation and save any intermediates needed for gradient computation in memory
 - <b>Backward</b>: apply the chain rule to compute the gradient of the loss function with respect to the inputs </font><br>
impractical 비실용적인

# Neural Networks

<img src='./Lesson pic/4-86.png'>

# Full implementation of training a 2-layer Neural Network needs ~ 20 lines

In [2]:
import numpy as np
from numpy.random import randn

N,D_in, H, D_out = 64,1000,100,10
x,y = randn(N,D_in), randn(N, D_out)
w1, w2 = randn(D_in, H), randn(H, D_out)

for t in range(2000):
    h = 1/(1+np.exp(-x.dot(w1)))
    y_pred = h.dot(w2)
    loss = np.square(y_pred - y).sum()
    if (t+1) %100 ==0:
        print(t+1,loss)
    
    grad_y_pred = 2. * (y_pred - y)
    grad_w2 = h.T.dot(grad_y_pred)
    grad_h = grad_y_pred.dot(w2.T)
    grad_w1 = x.T.dot(grad_h * h * (1-h))
    
    w1 -= 1e-4 * grad_w1
    w2 -= 1e-4 * grad_w2

100 2383.3188180028683
200 984.1325759813187
300 514.4370355837139
400 303.9939030338178
500 195.3180066338283
600 134.63161546130448
700 97.68503941709682
800 72.65913563294704
900 54.99697183816089
1000 42.08702861850945
1100 32.85345755416939
1200 26.162750447579246
1300 21.112771917681698
1400 17.18193342418582
1500 14.040825425303332
1600 11.543970856421858
1700 9.54495968421416
1800 7.9305878456433
1900 6.623170338536777
2000 5.555739589846581


<img src='./Lesson pic/4-94.png'>

# Be very careful with your brain analogies!
## Biological Neurons:
 - Many different types
 - Dendrites can perform complex non-linear computations
 - Synapses are not a single weight but a complex non-linear dynamical system
 - Rate code may not be adequate

<img src='./Lesson pic/4-96.png'>

# Summary
 - We arrange neurons into fully-connected layers
 - The abstraction of a layer has the nice property that it allows us to use efficient vectorized code (e.g. matrix multiplies)
 - Neural networks are not really neural
 - Next Time: Convolution Neural Network